In [1]:

import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
LANGSMITH_TRACING = os.getenv("LANGSMITH_TRACING")
LANGSMITH_ENDPOINT = os.getenv("LANGSMITH_ENDPOINT")
LANGSMITH_API_KEY = os.getenv("LANGSMITH_API_KEY")
LANGSMITH_PROJECT = os.getenv("LANGSMITH_PROJECT")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
# TRAVILY_PROJECT = os.getenv("TRAVILY_PROJECT")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGSMITH_TRACING"] = os.getenv("LANGSMITH_TRACING")
os.environ["LANGSMITH_ENDPOINT"] = os.getenv("LANGSMITH_ENDPOINT")
os.environ["LANGSMITH_PROJECT"] = os.getenv("LANGSMITH_PROJECT")

os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")



In [4]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest")

In [6]:
llm.invoke("hi")

AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-pro-002', 'safety_ratings': []}, id='run--7d4757bb-695a-4717-9f42-83e717beb7c3-0', usage_metadata={'input_tokens': 1, 'output_tokens': 11, 'total_tokens': 12, 'input_token_details': {'cache_read': 0}})

till now we ere using tool and creating agensts with the predefined thngs in langchaiin such  as react self ask tooll calling these all were predefinsed inthe langchain library, now we will learn to create agents from scratch so here we go

In [7]:
class Chatbot:
    def __init__(self,system=""):
        self.system=system
        self.message=[]
        if self.system:
            self.message.append({"role":"system","content":system})
    def __call__(self,message):
        self.message.append({"role":"user","content":message})
        result=self.execute()
        self.message.append({"role":"assistant","content":result})
        return result
        
    def execute(self):
        llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest")
        result = llm.invoke(self.message)
        return result.content

In [8]:

bot=Chatbot("you are a helpful assistant")

In [9]:

bot("hi")

'Hi there! How can I help you today?'

In [10]:
bot.message

[{'role': 'system', 'content': 'you are a helpful assistant'},
 {'role': 'user', 'content': 'hi'},
 {'role': 'assistant', 'content': 'Hi there! How can I help you today?'}]

the below regulart expression function is created for the identification of action term.

Understand the Regular expression
Pattern Breakdown:

^: This matches the start of a string. It means the string must begin with what follows.
Action:: This is a literal match. It means the string must have the text "Action:" at the beginning.
(\w+):
The parentheses () define a capture group. This allows you to extract part of the string that matches this section.
\w+ matches one or more word characters (letters, digits, and underscores). This will capture a word that follows "Action:".
:: This is a literal colon that separates the word matched by (\w+) from the rest of the string.
(.*):
This is another capture group, where .* matches any character (.) zero or more times (*), which means it captures everything that comes after the second colon.
What does it do?
This regex is looking for a string that:

Starts with the text "Action:".
Has a word right after it, separated by a colon.
Then, after another colon, it captures everything that follows.

In [15]:
import re
action_re = re.compile('^Action: (\w+): (.*)')

In [27]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop your output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you..
Observation will be the result of running those actions.


Your available actions are:
calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point
syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

simon_blog_search:
e.g. simon_blog_search: Python
Search Simon's blog for that term

Example session:
Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:
Observation: France is a country. The capital is Paris.

You then output:
Answer: The capital of France is Paris

Please Note: if you get basic conversation questions like "hi","hello","how are you?",\n
you have to answer "hi","hello","i am good".
""".strip()

In [28]:
import httpx
def wikipedia(query):
    response = httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": query,
        "format": "json"
    })
    return response.json()["query"]["search"][0]["snippet"]

In [29]:
import httpx
def simon_blog_search(query):
    response = httpx.get("https://datasette.simonwillison.net/simonwillisonblog.json", params={
        "sql": """
        select
          blog_entry.title || ': ' || substr(html_strip_tags(blog_entry.body), 0, 1000) as text,
          blog_entry.created
        from
          blog_entry join blog_entry_fts on blog_entry.rowid = blog_entry_fts.rowid
        where
          blog_entry_fts match escape_fts(:q)
        order by
          blog_entry_fts.rank
        limit
          1
        """.strip(),
        "_shape": "array",
        "q": query,
    })
    return response.json()[0]["text"]

In [30]:

def calculate(number):
    return eval(number)

In [35]:
known_actions = {
    "wikipedia": wikipedia,
    "calculate": calculate,
    "simon_blog_search": simon_blog_search
}

In [31]:
def query(question,max_turns=5):
    i = 0
    bot = Chatbot(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = f"Observation: {observation}"
        else:
            return result

In [32]:
bot = Chatbot(prompt)

In [33]:
bot("how are you??")

'Answer: i am good'

In [34]:
bot("what is your job?")

'Thought: I need to describe my job.\n\nAction: None\n\nPAUSE\n\nAnswer: I am a large language model, trained to provide helpful and informative responses to a wide range of queries and tasks.  I can generate text, translate languages, write different kinds of creative content, and answer your questions in an informative way, even if they are open ended, challenging, or strange.'

In [36]:
query("hi")

Answer: hi


'Answer: hi'

In [37]:
query("What is the capital of France?")

Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE
 -- running wikipedia France
Observation: <span class="searchmatch">France</span>, officially the <span class="searchmatch">French</span> Republic, is a country primarily located in Western Europe. Its overseas regions and territories include <span class="searchmatch">French</span> Guiana in
Answer: The capital of France is Paris.


'Answer: The capital of France is Paris.'

In [38]:
query("five plus five   ")

Thought: I need to calculate 5 + 5.
Action: calculate: 5 + 5
PAUSE
 -- running calculate 5 + 5
Observation: 10
Answer: Five plus five is 10.


'Answer: Five plus five is 10.'

In [39]:
query("has simon written over ai")

Thought: I should search Simon's blog for "AI".
Action: simon_blog_search: AI
PAUSE
 -- running simon_blog_search AI
Observation: Talking AI and jobs with Natasha Zouves for News Nation: I was interviewed by News Nation's Natasha Zouves about the very complicated topic of how we should think about AI in terms of threatening our jobs and careers. I previously talked with Natasha two years ago about Microsoft Bing.

I'll be honest: I was nervous about this one. I'm not an economist and I didn't feel confident talking about this topic!

I do find the challenge of making recent advances in AI and LLMs accessible to a general audience absolutely fascinating though, so I took the risk and agreed to the interview.

I think it came out very well. The full hour long video is now available on the News Nation YouTube channel, or as an audio podcast on iTunes or on Spotify.

 

I made my own transcript of the video (using MacWhisper) and fed it into the new Claude Opus 4 model to see if it could d

'Answer: Yes, Simon has written about AI on his blog, including an interview he did with News Nation about AI and jobs.  He also used AI tools to help process the transcript of the interview.'